**Distiller Installation on Drive**

In [15]:
from google.colab import drive
ROOT="/content/drive"
print(ROOT)
drive.mount(ROOT)

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Clone the git repo on your drive
!git clone https://github.com/NervanaSystems/distiller.git

I had created a folder named 'distiller' already. Realized later that it was of no need. 
So you would probably require this: %cd drive/My Drive/GE Edge Capstone 2020/distiller.  

In [2]:
%cd drive/My Drive/GE Edge Capstone 2020/distiller/distiller

/content/drive/My Drive/GE Edge Capstone 2020/distiller/distiller


In [ ]:
!pip3 install -e .

In [1]:
#Checking if Distiller is installed
import distiller

In [2]:
%pwd

'/content'

In [3]:
%cd drive/My Drive/GE Edge Capstone 2020/distiller/distiller/examples/classifier_compression

/content/drive/My Drive/GE Edge Capstone 2020/distiller/distiller/examples/classifier_compression


**Loading the CIFAR-10 Dataset**

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

import matplotlib.pyplot as plt
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset_CIFER = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

Files already downloaded and verified


'\ntrainloader_CIFER = torch.utils.data.DataLoader(trainset_CIFER, batch_size=4,\n                                          shuffle=True, num_workers=2)\n'

In [ ]:
#Cites all the Arguments which could be passed
!python3 compress_classifier.py --help

**Post Quantization Training** 

A list of architecture is defined for a specific dataset under the 'models' folder. Currently Distiller only supports Imagenet, CIFAR-10 and MNIST datasets. Imagenet however is not publicly available so loading the dataset is tedious. In order to experiment with architectures and datasets beyond distiller we would have to modify the codes, starting with 'models' and 'dataset loaders'. 'Modules' for distiller have also been defined to enable us to create our own architectures as per distiller (slight variation from pytorch)

In [ ]:
#Removed the pretrained argument which was given in the original command as received an error that the particular pretrained model is not available
!python3 compress_classifier.py -a resnet20_cifar './data'  --quantize-eval --evaluate

In [ ]:
#Removed the pretrained argument
!python3 compress_classifier.py -a plain20_cifar './data'  --quantize-eval --evaluate

**Quantization Aware Training**

'compress_classifier.py' is the common file used across all image classification problems. Different quantization techniques are applied through the compression scheduler which is saved as .yaml files. Besides varying the arguments from the command line, we can also make edits to the compression scheduler i.e. '.yaml' files, such as 'bits_activation', 'weights_activation', 'signed or unsigned weights' etc. This varies across schedulers. 

Path to access different compression schedulers built for QAT:
Capstone Project/distiller/examples/quantization/quant_aware_train



In [ ]:
# 1. Quant_aware_train_linear_quant.yaml 
# Read more at : "https://nervanasystems.github.io/distiller/algo_quantization.html"
#Removed the argument 'pretrained'
!python compress_classifier.py -a resnet20_cifar -p 50 -b 256 './data' --epochs 10 --compress=../quantization/quant_aware_train/quant_aware_train_linear_quant.yaml -j 22 --lr 0.0001 --vs 0 --gpu 0

In [ ]:
# 2. preact_resnet_cifar_dorefa.yaml
#Pre-activation ResNet20 on CIFAR10 (base FP32 + DoReFa)
#Changed epochs to 10 from 200
!python compress_classifier.py -a preact_resnet20_cifar --lr 0.1 -p 50 -b 128 './data' -j 1 --epochs 10 --compress=../quantization/quant_aware_train/preact_resnet_cifar_dorefa.yaml --wd=0.0002 --vs=0 --gpus 0